In [1]:
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

In [2]:
# download test file in here
# http://ai.stanford.edu/~amaas/data/sentiment
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'
labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:24


In [3]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index = False, encoding='utf-8')

In [4]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
df.shape

(50000, 2)

# 단어들을 feature vector로

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is hining', 'The weather is sweet', 'The sun is shining, the weather is sweet,'
                 'and one and one is two'])
bag = count.fit_transform(docs)

In [7]:
print(count.vocabulary_)

{'the': 7, 'sun': 5, 'is': 2, 'hining': 1, 'weather': 9, 'sweet': 6, 'shining': 4, 'and': 0, 'one': 3, 'two': 8}


In [8]:
print(bag.toarray())
#CountVectorizer에 들어있는 단어 사전의 유무로 0, 1을 표시한다

[[0 1 1 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 1 1 0 1]
 [2 0 3 2 1 1 1 2 1 1]]


# feature vector를 통한 중요도 분석 - tf-idf 기법

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())
# 즉 1번째 단어 sun이 feature vector에서 제일 중요도가 높다고 볼 수 있음
# 빈도수만 보면 is가 제일 높아야하지만, 첫번째, 두번째, 세번째 문장도 나오므로 tf-idf 기법을 통한 결과값은 낮음

[[0.   0.66 0.39 0.   0.   0.5  0.   0.39 0.   0.  ]
 [0.   0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.   0.44 0.5  0.25 0.19 0.19 0.29 0.25 0.19]]


# text data의 전처리

In [10]:
df.loc[0, 'review'][-50:]
# 데이터가 html 형식임

'is seven.<br /><br />Title (Brazil): Not Available'

In [11]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [12]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [13]:
preprocessor("</a>This :) is :( a test :-) ! ")
# 아직 이모티콘 형태의 문자가 안사라졌으므로 따로 전처리를 진행해주어야 한다.

'this is a test :) :( :)'

In [14]:
df['review'] = df['review'].apply(preprocessor)

# Tokenizing

In [15]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running asn thus they run')
# 스페이스 단위로 tokenizing

['runners', 'like', 'running', 'asn', 'thus', 'they', 'run']

In [16]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

# 앞의 실행과는 다르게 running이 run으로 tokenizing됨

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

# stopwords는 별다른 정보를 포함하지않을 확률이 높은 단어, 즉 is, has, like 같은거
# nltk통해 다운로드 받아 전처리 할때 제거 가능

['runner', 'like', 'run', 'run', 'lot']

# 로지스틱 회귀를 통한 문서분류

In [19]:
X_train = df.loc[:700, 'review'].values
y_train = df.loc[:700, 'sentiment'].values
X_test = df.loc[700:1000, 'review'].values
y_test = df.loc[700:1000, 'sentiment'].values

# 본문은거의 몇 만개이나 , 몇만개 돌리면 작업이 너무 느림

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range' : [(1, 1)],
               'vect__stop_words' : [stop, None],
               'vect__tokenizer' : [tokenizer, tokenizer_porter],
               'clf__penalty' : ['l1', 'l2'],
               'clf__C' : [1.0, 10.0, 100.0]},
              {'vect__ngram_range' : [(1, 1)],
                'vect__stop_words' : [stop, None],
                'vect__tokenizer' : [tokenizer, tokenizer_porter],
                'vect__use_idf' : [False],
                'vect__norm' : [None],
                'clf__penalty' : ['l1', 'l2'],
                'clf__C' : [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv = 5, n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [21]:
print('Best parameter set: %s' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each

In [22]:
print('CV Accuracy : %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy : 0.840


In [23]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy : %.3f' % clf.score(X_test, y_test))

Test Accuracy : 0.811


# Out of core learning

In [24]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [25]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [26]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [27]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [28]:
# SGD분류기를 학습할때에는 메모리상 완벽한 단어를 사용하여야하므로, 전처리에서 HashingVectorizer를 사용함
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21,
                         preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [29]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size = 1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

# mini_batch의 개수는 45개이고, 1개의 batch당 1000개로 45000개의 데이터를 학습하였는데, 위에 로지스틱 회귀 grid search를 1000개만 한것보다 훨씬 빠르다

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


In [30]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy : %.3f' % clf.score(X_test, y_test))

Accuracy : 0.868


In [31]:
clf = clf.partial_fit(X_test, y_test)

# Topic Modeling with Latent Dirichlet Allocation

In [32]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

In [33]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

# text tokenizing을 통해 비지도학습 실행

In [34]:
lda.components_.shape

(10, 5000)

In [35]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print('Topic %d:' % (topic_idx + 1))
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                    [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [36]:
# 즉 topic 5에 포함된 단어들이 많으면 비슷한 종류의 영화일 확률이 높고,
# 이를 잠재디리클레 할당으로 확률적으로 계산.
# 이중 확률이 높은 5개의 영화 리뷰를 출력
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d' % (iter_idx+1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [37]:
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)
# 앞선 Ch8에서 clf와 stop 객체를 pkl형태로 저장
# Ch9 파이썬 웹 구동 위해 pickle형태로 객체 저장